In [4]:
import gensim
import pandas as pd
import numpy as np

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('all.bin', binary=True)

In [6]:
def read_data(file):
    with open(file) as f:
        sentences = [sentence.split('\n') for sentence in f.read().split('\n\n')]
        df = pd.DataFrame([line.split() for sentence in sentences for line in sentence], columns=['word', 'tag'])
        df['tag'] = df['tag'].map(change_tag)
    return df


def change_tag(tag):
    if tag == 'O':
        return tag
    tag = tag[2:]
    return tag

In [21]:
train = read_data('data/collection_3/train_sa.txt')
valid = read_data('data/collection_3/valid_sa.txt')
test = read_data('data/collection_3/test_sa.txt')

In [8]:
train['tag'].value_counts()

O      133250
PER     11433
ORG      6096
LOC      4360
Name: tag, dtype: int64

In [9]:
test['tag'].value_counts()

O      28405
PER     2590
ORG     1307
LOC     1026
Name: tag, dtype: int64

In [28]:
def get_embedding(word):
    try:
        return model.get_vector(word.lower()).tolist()
    except KeyError:
        return np.zeros(150).tolist()
    

def get_features(df):
    X = pd.DataFrame(df['word'].apply(lambda x: get_embedding(x)).tolist())
    X['word'] = df['word'].map(lambda x: ' '.join(x))
    X['cap'] = X['word'].map(lambda x: x.istitle())
    X['alpha'] = X['word'].map(lambda x: x.isalpha())
    return X

In [29]:
X_train = get_features(train)
X_valid = get_features(valid)
X_test = get_features(test)

y_train = train['tag']
y_valid = valid['tag']
y_test = test['tag']

In [25]:
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, f1_score

In [34]:
clf = CatBoostClassifier(iterations=10000, task_type='GPU')

In [35]:
clf.fit(X_train, y_train, text_features=['word'], eval_set=(X_valid, y_valid), use_best_model=True, early_stopping_rounds=10)

Learning rate set to 0.063987
0:	learn: 1.2458455	test: 1.2432762	best: 1.2432762 (0)	total: 19.9ms	remaining: 3m 19s
1:	learn: 1.1343088	test: 1.1295961	best: 1.1295961 (1)	total: 34.4ms	remaining: 2m 51s
2:	learn: 1.0432859	test: 1.0369071	best: 1.0369071 (2)	total: 47.3ms	remaining: 2m 37s
3:	learn: 0.9670181	test: 0.9589070	best: 0.9589070 (3)	total: 62.3ms	remaining: 2m 35s
4:	learn: 0.9017302	test: 0.8926726	best: 0.8926726 (4)	total: 77.1ms	remaining: 2m 34s
5:	learn: 0.8454434	test: 0.8354443	best: 0.8354443 (5)	total: 93.4ms	remaining: 2m 35s
6:	learn: 0.7964836	test: 0.7856080	best: 0.7856080 (6)	total: 108ms	remaining: 2m 33s
7:	learn: 0.7532099	test: 0.7419414	best: 0.7419414 (7)	total: 122ms	remaining: 2m 32s
8:	learn: 0.7151744	test: 0.7035241	best: 0.7035241 (8)	total: 137ms	remaining: 2m 31s
9:	learn: 0.6810275	test: 0.6689777	best: 0.6689777 (9)	total: 151ms	remaining: 2m 31s
10:	learn: 0.6503095	test: 0.6379523	best: 0.6379523 (10)	total: 166ms	remaining: 2m 30s
11:	l

95:	learn: 0.2889828	test: 0.2785846	best: 0.2785846 (95)	total: 1.23s	remaining: 2m 7s
96:	learn: 0.2884026	test: 0.2780166	best: 0.2780166 (96)	total: 1.25s	remaining: 2m 7s
97:	learn: 0.2880578	test: 0.2776536	best: 0.2776536 (97)	total: 1.26s	remaining: 2m 7s
98:	learn: 0.2876894	test: 0.2773811	best: 0.2773811 (98)	total: 1.27s	remaining: 2m 7s
99:	learn: 0.2874157	test: 0.2770995	best: 0.2770995 (99)	total: 1.28s	remaining: 2m 7s
100:	learn: 0.2869978	test: 0.2767682	best: 0.2767682 (100)	total: 1.3s	remaining: 2m 7s
101:	learn: 0.2867401	test: 0.2764871	best: 0.2764871 (101)	total: 1.31s	remaining: 2m 6s
102:	learn: 0.2865412	test: 0.2763303	best: 0.2763303 (102)	total: 1.32s	remaining: 2m 6s
103:	learn: 0.2862058	test: 0.2759551	best: 0.2759551 (103)	total: 1.33s	remaining: 2m 6s
104:	learn: 0.2858973	test: 0.2756896	best: 0.2756896 (104)	total: 1.34s	remaining: 2m 6s
105:	learn: 0.2855324	test: 0.2753055	best: 0.2753055 (105)	total: 1.35s	remaining: 2m 5s
106:	learn: 0.2852839

187:	learn: 0.2694746	test: 0.2599274	best: 0.2599274 (187)	total: 2.25s	remaining: 1m 57s
188:	learn: 0.2693170	test: 0.2597562	best: 0.2597562 (188)	total: 2.26s	remaining: 1m 57s
189:	learn: 0.2691267	test: 0.2595630	best: 0.2595630 (189)	total: 2.27s	remaining: 1m 57s
190:	learn: 0.2690402	test: 0.2594949	best: 0.2594949 (190)	total: 2.28s	remaining: 1m 57s
191:	learn: 0.2689492	test: 0.2594205	best: 0.2594205 (191)	total: 2.29s	remaining: 1m 57s
192:	learn: 0.2687038	test: 0.2591152	best: 0.2591152 (192)	total: 2.3s	remaining: 1m 57s
193:	learn: 0.2683429	test: 0.2588429	best: 0.2588429 (193)	total: 2.32s	remaining: 1m 57s
194:	learn: 0.2681509	test: 0.2586651	best: 0.2586651 (194)	total: 2.33s	remaining: 1m 57s
195:	learn: 0.2680806	test: 0.2586144	best: 0.2586144 (195)	total: 2.34s	remaining: 1m 57s
196:	learn: 0.2679191	test: 0.2584589	best: 0.2584589 (196)	total: 2.35s	remaining: 1m 56s
197:	learn: 0.2677506	test: 0.2583312	best: 0.2583312 (197)	total: 2.36s	remaining: 1m 56s


289:	learn: 0.2594218	test: 0.2508834	best: 0.2508834 (289)	total: 3.25s	remaining: 1m 48s
290:	learn: 0.2593667	test: 0.2508603	best: 0.2508603 (290)	total: 3.26s	remaining: 1m 48s
291:	learn: 0.2592686	test: 0.2507652	best: 0.2507652 (291)	total: 3.27s	remaining: 1m 48s
292:	learn: 0.2592483	test: 0.2507358	best: 0.2507358 (292)	total: 3.28s	remaining: 1m 48s
293:	learn: 0.2592273	test: 0.2507347	best: 0.2507347 (293)	total: 3.29s	remaining: 1m 48s
294:	learn: 0.2591762	test: 0.2506792	best: 0.2506792 (294)	total: 3.3s	remaining: 1m 48s
295:	learn: 0.2590080	test: 0.2505819	best: 0.2505819 (295)	total: 3.31s	remaining: 1m 48s
296:	learn: 0.2588633	test: 0.2504507	best: 0.2504507 (296)	total: 3.32s	remaining: 1m 48s
297:	learn: 0.2588196	test: 0.2504210	best: 0.2504210 (297)	total: 3.33s	remaining: 1m 48s
298:	learn: 0.2587830	test: 0.2503722	best: 0.2503722 (298)	total: 3.33s	remaining: 1m 48s
299:	learn: 0.2587213	test: 0.2503388	best: 0.2503388 (299)	total: 3.34s	remaining: 1m 48s


399:	learn: 0.2527666	test: 0.2456547	best: 0.2456547 (399)	total: 4.26s	remaining: 1m 42s
400:	learn: 0.2526601	test: 0.2455899	best: 0.2455899 (400)	total: 4.27s	remaining: 1m 42s
401:	learn: 0.2526437	test: 0.2455883	best: 0.2455883 (401)	total: 4.28s	remaining: 1m 42s
402:	learn: 0.2526111	test: 0.2455643	best: 0.2455643 (402)	total: 4.29s	remaining: 1m 42s
403:	learn: 0.2525956	test: 0.2455565	best: 0.2455565 (403)	total: 4.3s	remaining: 1m 42s
404:	learn: 0.2525823	test: 0.2455523	best: 0.2455523 (404)	total: 4.31s	remaining: 1m 42s
405:	learn: 0.2525430	test: 0.2455092	best: 0.2455092 (405)	total: 4.32s	remaining: 1m 41s
406:	learn: 0.2524992	test: 0.2454814	best: 0.2454814 (406)	total: 4.32s	remaining: 1m 41s
407:	learn: 0.2524145	test: 0.2454109	best: 0.2454109 (407)	total: 4.33s	remaining: 1m 41s
408:	learn: 0.2523840	test: 0.2453963	best: 0.2453963 (408)	total: 4.34s	remaining: 1m 41s
409:	learn: 0.2523344	test: 0.2453127	best: 0.2453127 (409)	total: 4.35s	remaining: 1m 41s


...

2180:	learn: 0.2244279	test: 0.2280805	best: 0.2280805 (2180)	total: 20.8s	remaining: 1m 14s
2181:	learn: 0.2244172	test: 0.2280690	best: 0.2280690 (2181)	total: 20.8s	remaining: 1m 14s
2182:	learn: 0.2244107	test: 0.2280691	best: 0.2280690 (2181)	total: 20.8s	remaining: 1m 14s
2183:	learn: 0.2244044	test: 0.2280650	best: 0.2280650 (2183)	total: 20.8s	remaining: 1m 14s
2184:	learn: 0.2244014	test: 0.2280623	best: 0.2280623 (2184)	total: 20.8s	remaining: 1m 14s
2185:	learn: 0.2243978	test: 0.2280619	best: 0.2280619 (2185)	total: 20.8s	remaining: 1m 14s
2186:	learn: 0.2243836	test: 0.2280606	best: 0.2280606 (2186)	total: 20.9s	remaining: 1m 14s
2187:	learn: 0.2243805	test: 0.2280579	best: 0.2280579 (2187)	total: 20.9s	remaining: 1m 14s
2188:	learn: 0.2243486	test: 0.2280514	best: 0.2280514 (2188)	total: 20.9s	remaining: 1m 14s
2189:	learn: 0.2243444	test: 0.2280500	best: 0.2280500 (2189)	total: 20.9s	remaining: 1m 14s
2190:	learn: 0.2243405	test: 0.2280411	best: 0.2280411 (2190)	total: 2

2270:	learn: 0.2237525	test: 0.2277783	best: 0.2277783 (2270)	total: 21.6s	remaining: 1m 13s
2271:	learn: 0.2237483	test: 0.2277744	best: 0.2277744 (2271)	total: 21.6s	remaining: 1m 13s
2272:	learn: 0.2237468	test: 0.2277727	best: 0.2277727 (2272)	total: 21.6s	remaining: 1m 13s
2273:	learn: 0.2237401	test: 0.2277730	best: 0.2277727 (2272)	total: 21.6s	remaining: 1m 13s
2274:	learn: 0.2237236	test: 0.2277705	best: 0.2277705 (2274)	total: 21.6s	remaining: 1m 13s
2275:	learn: 0.2237147	test: 0.2277642	best: 0.2277642 (2275)	total: 21.7s	remaining: 1m 13s
2276:	learn: 0.2236955	test: 0.2277545	best: 0.2277545 (2276)	total: 21.7s	remaining: 1m 13s
2277:	learn: 0.2236900	test: 0.2277512	best: 0.2277512 (2277)	total: 21.7s	remaining: 1m 13s
2278:	learn: 0.2236835	test: 0.2277465	best: 0.2277465 (2278)	total: 21.7s	remaining: 1m 13s
2279:	learn: 0.2236766	test: 0.2277258	best: 0.2277258 (2279)	total: 21.7s	remaining: 1m 13s
2280:	learn: 0.2236695	test: 0.2277192	best: 0.2277192 (2280)	total: 2

2376:	learn: 0.2230093	test: 0.2274430	best: 0.2274418 (2375)	total: 22.6s	remaining: 1m 12s
2377:	learn: 0.2230012	test: 0.2274384	best: 0.2274384 (2377)	total: 22.6s	remaining: 1m 12s
2378:	learn: 0.2229997	test: 0.2274385	best: 0.2274384 (2377)	total: 22.6s	remaining: 1m 12s
2379:	learn: 0.2229867	test: 0.2274387	best: 0.2274384 (2377)	total: 22.6s	remaining: 1m 12s
2380:	learn: 0.2229719	test: 0.2274331	best: 0.2274331 (2380)	total: 22.7s	remaining: 1m 12s
2381:	learn: 0.2229555	test: 0.2274190	best: 0.2274190 (2381)	total: 22.7s	remaining: 1m 12s
2382:	learn: 0.2229430	test: 0.2274143	best: 0.2274143 (2382)	total: 22.7s	remaining: 1m 12s
2383:	learn: 0.2229372	test: 0.2274122	best: 0.2274122 (2383)	total: 22.7s	remaining: 1m 12s
2384:	learn: 0.2229348	test: 0.2274135	best: 0.2274122 (2383)	total: 22.7s	remaining: 1m 12s
2385:	learn: 0.2229311	test: 0.2274135	best: 0.2274122 (2383)	total: 22.7s	remaining: 1m 12s
2386:	learn: 0.2229216	test: 0.2274174	best: 0.2274122 (2383)	total: 2

2467:	learn: 0.2223750	test: 0.2271615	best: 0.2271612 (2466)	total: 23.4s	remaining: 1m 11s
2468:	learn: 0.2223627	test: 0.2271512	best: 0.2271512 (2468)	total: 23.4s	remaining: 1m 11s
2469:	learn: 0.2223589	test: 0.2271529	best: 0.2271512 (2468)	total: 23.4s	remaining: 1m 11s
2470:	learn: 0.2223578	test: 0.2271516	best: 0.2271512 (2468)	total: 23.4s	remaining: 1m 11s
2471:	learn: 0.2223429	test: 0.2271543	best: 0.2271512 (2468)	total: 23.5s	remaining: 1m 11s
2472:	learn: 0.2223346	test: 0.2271497	best: 0.2271497 (2472)	total: 23.5s	remaining: 1m 11s
2473:	learn: 0.2223203	test: 0.2271359	best: 0.2271359 (2473)	total: 23.5s	remaining: 1m 11s
2474:	learn: 0.2223080	test: 0.2271284	best: 0.2271284 (2474)	total: 23.5s	remaining: 1m 11s
2475:	learn: 0.2223055	test: 0.2271274	best: 0.2271274 (2475)	total: 23.5s	remaining: 1m 11s
2476:	learn: 0.2222967	test: 0.2271318	best: 0.2271274 (2475)	total: 23.5s	remaining: 1m 11s
2477:	learn: 0.2222850	test: 0.2271199	best: 0.2271199 (2477)	total: 2

2558:	learn: 0.2217139	test: 0.2268984	best: 0.2268984 (2558)	total: 24.2s	remaining: 1m 10s
2559:	learn: 0.2217101	test: 0.2268908	best: 0.2268908 (2559)	total: 24.2s	remaining: 1m 10s
2560:	learn: 0.2217052	test: 0.2268937	best: 0.2268908 (2559)	total: 24.2s	remaining: 1m 10s
2561:	learn: 0.2217002	test: 0.2268875	best: 0.2268875 (2561)	total: 24.3s	remaining: 1m 10s
2562:	learn: 0.2216969	test: 0.2268833	best: 0.2268833 (2562)	total: 24.3s	remaining: 1m 10s
2563:	learn: 0.2216913	test: 0.2268836	best: 0.2268833 (2562)	total: 24.3s	remaining: 1m 10s
2564:	learn: 0.2216643	test: 0.2268767	best: 0.2268767 (2564)	total: 24.3s	remaining: 1m 10s
2565:	learn: 0.2216590	test: 0.2268754	best: 0.2268754 (2565)	total: 24.3s	remaining: 1m 10s
2566:	learn: 0.2216564	test: 0.2268739	best: 0.2268739 (2566)	total: 24.3s	remaining: 1m 10s
2567:	learn: 0.2216498	test: 0.2268678	best: 0.2268678 (2567)	total: 24.3s	remaining: 1m 10s
2568:	learn: 0.2216482	test: 0.2268680	best: 0.2268678 (2567)	total: 2

In [37]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         LOC       0.74      0.61      0.67      1026
           O       0.94      0.98      0.96     28405
         ORG       0.79      0.41      0.54      1307
         PER       0.81      0.65      0.72      2590

    accuracy                           0.92     33328
   macro avg       0.82      0.66      0.72     33328
weighted avg       0.91      0.92      0.91     33328



In [33]:
print(classification_report(y_test, clf.predict(X_test)))

              precision    recall  f1-score   support

         LOC       0.74      0.61      0.67      1026
           O       0.93      0.98      0.96     28405
         ORG       0.77      0.39      0.51      1307
         PER       0.81      0.65      0.72      2590

    accuracy                           0.92     33328
   macro avg       0.81      0.65      0.71     33328
weighted avg       0.91      0.92      0.91     33328



In [42]:
list(zip(test['word'], test['tag'], clf.predict(X_test)[:, 0]))

[('А', 'PER', 'PER'),
 ('.', 'PER', 'O'),
 ('Силуанов', 'PER', 'PER'),
 ('Арассыыйаттан', 'LOC', 'O'),
 ('ИВФ', 'O', 'O'),
 ('-', 'O', 'O'),
 ('га', 'O', 'O'),
 ('управляющайынан', 'O', 'O'),
 ('ананна', 'O', 'O'),
 ('Россия', 'LOC', 'O'),
 ('Президенэ', 'O', 'O'),
 ('Дмитрий', 'PER', 'PER'),
 ('Медведев', 'O', 'PER'),
 ('ыйааҕынан', 'O', 'O'),
 ('РФ', 'LOC', 'LOC'),
 ('үбүн', 'O', 'O'),
 ('миниистирэ', 'O', 'O'),
 ('Антон', 'PER', 'PER'),
 ('Силуанов', 'O', 'PER'),
 ('аан', 'O', 'O'),
 ('дойдутааҕы', 'O', 'O'),
 ('валютнай', 'O', 'O'),
 ('фондаҕа', 'O', 'O'),
 (',', 'O', 'O'),
 ('ону', 'O', 'O'),
 ('тэҥэ', 'O', 'O'),
 ('аан', 'O', 'O'),
 ('дойдутааҕы', 'O', 'O'),
 ('баан', 'O', 'O'),
 ('бөлөҕүн', 'O', 'O'),
 ('тэрилтэлэригэр', 'O', 'O'),
 ('анаата', 'O', 'O'),
 ('.', 'O', 'O'),
 ('"', 'O', 'O'),
 ('РФ', 'LOC', 'LOC'),
 ('үбүн', 'O', 'O'),
 ('миниистирэ', 'O', 'O'),
 ('А', 'PER', 'PER'),
 ('.', 'PER', 'O'),
 ('Силуанова', 'PER', 'PER'),
 ('РФ', 'LOC', 'LOC'),
 ('-', 'O', 'O'),
 ('тан',

In [44]:
list(zip(train['word'], train['tag'], clf.predict(X_train)[:, 0]))

[('Эбии', 'O', 'O'),
 (':', 'O', 'O'),
 ('Д', 'PER', 'PER'),
 ('.', 'PER', 'O'),
 ('Медведев', 'PER', 'PER'),
 ('полиция', 'O', 'O'),
 ('үлэһиттэрин', 'O', 'O'),
 ('аатын', 'O', 'O'),
 ('-', 'O', 'O'),
 ('суолун', 'O', 'O'),
 ('ылан', 'O', 'O'),
 ('РФ', 'O', 'LOC'),
 ('ИДьМ', 'ORG', 'ORG'),
 (',', 'O', 'O'),
 ('ГУВД', 'ORG', 'ORG'),
 (',', 'O', 'O'),
 ('ИДьМ', 'O', 'ORG'),
 ('субъектарыгар', 'O', 'O'),
 ('ИДьМ', 'O', 'ORG'),
 ('салайааччыларын', 'O', 'O'),
 ('ыҥырда', 'O', 'O'),
 ('.', 'O', 'O'),
 ('Россия', 'LOC', 'O'),
 ('Президенэ', 'O', 'O'),
 ('Дмитрий', 'PER', 'PER'),
 ('Медведев', 'PER', 'PER'),
 ('полиция', 'O', 'O'),
 ('үлэһиттэрин', 'O', 'O'),
 ('аатын', 'O', 'O'),
 ('иҥэрдэ', 'O', 'O'),
 ('уонна', 'O', 'O'),
 ('РФ', 'LOC', 'LOC'),
 ('субъектарыгар', 'O', 'O'),
 ('уонна', 'O', 'O'),
 ('федеральнай', 'O', 'O'),
 ('уокуруктарга', 'O', 'O'),
 ('ис', 'O', 'O'),
 ('дьыала', 'O', 'O'),
 ('министерствотын', 'O', 'O'),
 ('14', 'O', 'O'),
 ('управлениеларыгар', 'O', 'O'),
 (',', 'O', 

In [44]:
list(zip(test['word'], clf.predict(test['emb'])[:, 0]))[:100]

[('А', 'O'),
 ('.', 'O'),
 ('Силуанов', 'PER'),
 ('Арассыыйаттан', 'O'),
 ('ИВФ', 'O'),
 ('-', 'O'),
 ('га', 'O'),
 ('управляющайынан', 'O'),
 ('ананна', 'O'),
 ('Россия', 'O'),
 ('Президенэ', 'O'),
 ('Дмитрий', 'PER'),
 ('Медведев', 'PER'),
 ('ыйааҕынан', 'O'),
 ('РФ', 'LOC'),
 ('үбүн', 'O'),
 ('миниистирэ', 'O'),
 ('Антон', 'PER'),
 ('Силуанов', 'PER'),
 ('аан', 'O'),
 ('дойдутааҕы', 'O'),
 ('валютнай', 'O'),
 ('фондаҕа', 'O'),
 (',', 'O'),
 ('ону', 'O'),
 ('тэҥэ', 'O'),
 ('аан', 'O'),
 ('дойдутааҕы', 'O'),
 ('баан', 'O'),
 ('бөлөҕүн', 'O'),
 ('тэрилтэлэригэр', 'O'),
 ('анаата', 'O'),
 ('.', 'O'),
 ('"', 'O'),
 ('РФ', 'LOC'),
 ('үбүн', 'O'),
 ('миниистирэ', 'O'),
 ('А', 'O'),
 ('.', 'O'),
 ('Силуанова', 'O'),
 ('РФ', 'LOC'),
 ('-', 'O'),
 ('тан', 'O'),
 ('аан', 'O'),
 ('дойдутааҕы', 'O'),
 ('валютнай', 'O'),
 ('фондаҕа', 'O'),
 (',', 'O'),
 ('инвестициялары', 'O'),
 ('мэктиэлээһиҥҥэ', 'O'),
 ('уонна', 'O'),
 ('киэҥ', 'O'),
 ('өрүттээх', 'O'),
 ('агентствоҕа', 'O'),
 ('үлэлиир', 'O'),